# Just Intonation

*mu* has a very advanced support of [Just Intonation](https://en.wikipedia.org/wiki/Just_intonation) through its submodule *mu.mel.ji*. The purpose of this module may range from experimenting with weird, new tunings to educational demonstrations of - let's say a [Pythagorean scale](https://en.wikipedia.org/wiki/Pythagorean_tuning). The goal of this Jupyter Notebook is to give you a basic overview of how *mu* abstracts Just Intonation and what functionality it offers.

## Basic concepts

Before we start, it is important to mention that *mu.mel.ji* doesn't make a difference between an interval and a pitch. The difference between both are made through the interpretation of the user. To make this statement clearer, an easy example may help: let's say you have a imaginary, boring scale with only two pitches and what you would like to find out is which interval divides these two pitches. So what you could do in *mu* is just subtracting one pitch from the other. The resulting object will be the interval which divdes them. But in the next step you could just append this resulting interval to your 2-pitch-scale to get a little bit more interesting 3-pitch-scale. In your new interpretation this interval between your two pitches is no longer this interval but a pitch of a 3-pitch scale! If this isn't 100% clear yet, don't worry: I'm sure its getting clearer - step by step - if we look at actual examples.

## Creating new intervals

First of all you have to import the ji-module in your Python interpreter:

In [2]:
from mu.mel import ji

To initialize a new Pitch-object, you could use the *r* (for *ratio*) - function. This function expects two arguments: one integer for the ratios numerator and one integer for its denominator. To create a [Perfect Fifth](https://en.wikipedia.org/wiki/Perfect_fifth) with a frequency ratio of 3/2, you just need to type:

In [2]:
ji.r(3, 2)

You could generate any Just Intonation interval that way. Furthermore you could also get new intervals through adding or subtracting two different intervals:

In [ ]:
perfect_fifth = ji.r(3, 2)
major_third = ji.r(5, 4)
major_seventh = perfect_fifth + major_third
minor_third = perfect_fifth - major_third
perfect_unison = perfect_fifth - perfect_fifth

It is also possible to add the same interval n-times through the scalar-method:

In [ ]:
ji.r(3, 2).scalar(2) == ji.r(3, 2) + ji.r(3, 2)
ji.r(3, 2).scalar(3) == ji.r(3, 2) + ji.r(3, 2) + ji.r(3, 2)

This way you could for instance generate a [Pythagorean scale](https://en.wikipedia.org/wiki/Pythagorean_tuning) which is defined as ["a system of musical tuning in which the frequency ratios of all intervals are based on the ratio 3:2."](https://en.wikipedia.org/wiki/Pythagorean_tuning). This example uses Pythons [List comprehension syntax](https://www.python-course.eu/python3_list_comprehension.php):

In [ ]:
pythagorean_scale = [ji.r(3, 2).scalar(i) for i in range(12)]

You may have noticed that through adding different intervals, it happens easily that the resulting ratios are higher than an octave (2/1). In classical western music theory octave equivalence is often assumed, which is why it's sometimes helpful to reduce all intervals to an octave. For this case *mu* offers the *normalize* - method. This method expects one argument, which has to be a positive integer. *mu* will interpret this integer *j* as the numerator of a ratio of the form j/1 and it will subtract the concering interval with the j/1 - interval until it is smaller than the j/1 - interval. If the concerning interval is smaller than 1/1 it will also add this interval as often with the j/1 interval until the concerning interval is bigger or equal to 1/1. This means that *normalize(2)* results in scales, which repeat after an octave, while *normalize(3)* may result in scales, which repeat after an octave plus a perfect fifth (an example for such a tuning would be the famous [Bohlen-Pierce-Scale](https://en.wikipedia.org/wiki/Bohlen%E2%80%93Pierce_scale)).

In [ ]:
pythagorean_scale_normalized = [ji_pitch.normalize(2) for ji_pitch in pythagorean_scale]

*mu* could also calculate you the complement of an interval:

In [ ]:
my_fourth = ji.r(3, 2).inverse()
my_fifth = ji.r(2, 3).inverse()
inverse_pythagorean_scale = [ji_pitch.inverse().normalize(2) for ji_pitch in pythagorean_scale]

## Inspecting intervals

While the default representations of Pitch-objects are ratios, it is also possible to return its ratio converted to a floating point number:

In [ ]:
ji.r(3, 2).float

You could find out which prime numbers occur in your interval:

In [ ]:
ji.r(77, 74).primes

*mu* also support different formulae for harmonic complexity like [Eulers](https://en.wikipedia.org/wiki/Leonhard_Euler) "gradus suvitatis", its adaption of the German musicologist Martin Vogel, James Tenney [Harmonic Distance](http://www.tonalsoft.com/enc/h/harmonic-distance.aspx), [Erv Wilsons](http://www.anaphoria.com/wilson.html) harmonicity calculation and last but not least Clarence Barlows awesome harmonicity formula. While most of these return harmonic complexity (meaning a higher return value equals a more dissonant sound), Barlows formula returns the consonance of an interval (meaning a higher return value equals a more consonant sound):

In [ ]:
minor_third0 = ji.r(6, 5)
minor_third1 = ji.r(7, 6)

minor_third0.harmonicity_euler
minor_third0.harmonicity_vogel
minor_third0.harmonicity_barlow
minor_third0.harmonicity_tenney
minor_third0.harmonicity_wilson

minor_third1.harmonicity_euler
minor_third1.harmonicity_vogel
minor_third1.harmonicity_barlow
minor_third1.harmonicity_tenney
minor_third1.harmonicity_wilson

If you try out Barlows harmonicity formula a couple of times, you may notice that the result is sometimes a positive float and sometimes a negative float. The reason for this is that intervals differs in - what I called - *gender*. This attribute of an interval describes, whether the highest occurring prime number of a pitch is located in its numerator or in its denominator. Since inversing an interval means swaping the numerator and the denominator, the inversion of a pitch has always the opposite gender of the asked pitch.

In [ ]:
ji.r(3, 2).gender  # gender is True since highest prime number (3) is located in the numerator
ji.r(3, 2).inverse().gender  # gender is False since the inversion has always the opposite gender
ji.r(26, 17).gender  # gender is True since highest prime number (17) is located in the denominator
ji.r(1, 1).gender  # gender is arbitrarily defined as True

## Understanding deeply *mu*s implementation of Just Intonation

### Monzos and vals

Until now we always created new pitches through the *ji.r* - function. If you may ask your pitch object for its type, your Python interpreter will return "JIPitch". 

In [ ]:
type(ji.r(3, 2))

This means, the *ji.r* - function is actually [syntactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar) for:

In [ ]:
ji.JIPitch([-1, 1]) == ji.r(3, 2)

But what does this two element list [-1, 1] mean? Why does the same Just Intonation pitch like *ji.r(3, 2)* result, if we enter this list to the JIPitch - class? To answer this question, we have to understand that *mu* actually doesn't implement Just Intonation with ratios but with [Monzos](http://xenharmonic.wikispaces.com/monzos). Monzos are a concept named (and perhaps designed) by the composer [Joe Monzo](http://xenharmonic.wikispaces.com/Joe+Monzo). While ratios might offer an easy and simple interface to enter fast new pitches or to distinguish them, Monzos will give you a comprehensive understanding of what really happens with your pitches. 
A Monzo could be understood as a vector, which contains exponents for prime numbers. The corresponding prime numbers are saved in a similar vector called "val". Hence, every pitch object contains a "val" - property. If the val of a pitch - object would be (2, 3, 5) and the Monzo of the Object would be (-2, 0, 1) the resulting     
interval in p/q - notation would be 5/4, since:              


In [ ]:
2**-2 * 3**0 * 5**1 == 5/4

If you write the monzo-vector straight over the val-vector the relationship between both becomes clear:             
(-2, 0, 1)                                                    
(2, 3, 5)   
If we return to our first example, it should be clear now: [-1, 1] is the Monzo to the corresponding Val [2, 3], so you actually calculate:

In [ ]:
2**-1 * 3**1 == 3/2

If you generate a JIPitch - object with the *ji.r* function, *mu* automatically converts the ratio to the corresponding Monzo for you.

We always looked at the pitch - objects through their ratio or their float representation, but of course it is also possible to inspect their monzo:

In [ ]:
for pitch in pythagorean_scale:
    print(pitch.monzo)

If you compare this representation with the complicated ratio-representation of the scale, you could be suprised how easy the structure of these pitches are! You see instantly how the scale has been generated through the falling first number and the rising second number in the Monzo (you remember, we only stacked many 3/2 through the *scalar* - method).

### The *val_border* - property
We could also check out our normalized scale:

In [ ]:
for pitch in pythagorean_scale_normalized:
    print(pitch.monzo)

Now you see that - while the second elements in the Monzos kept their structure, the first elements become a little bit weird. The reason for this is that due to the normalisation-process the number 2 has to be altered to realize a scale with only pitches in one octave. We could say, that the prime number 2 actually doesn't play an important role from this "classical music theory perspective". And because the prime number 2 isn't really relevant, we could tell *mu* that it should ignore this number. To be able to tell *mu* this, it offers for pitch - objects the *val_border* property. By default this property is always 1:

In [ ]:
print(ji.r(3, 2).val_border)

You could tell *mu* when you initialize	a new pitch object which *val_border* it should use:

In [ ]:
print(ji.r(3, 2, val_border=2).val_border)
print(ji.JIPitch([1], val_border=2).val_border)  # notice that you don't need the -1!

Or you could set the *val_border* of an existing pitch:

In [ ]:
my_sixth = ji.r(13, 8)
my_sixth.val_border = 2

While this method changes your actual pitch, *mu* offers last but not least a non destructive way to alter the *val_border*:

In [ ]:
my_sixth_with_val_border_1 = ji.r(13, 8)
my_sixth_with_val_border_3 = my_sixth_with_val_border_1.set_val_border(3)

If you inspect the *val* property of these pitches, you can see that it changes when you change the *val_border* - property:

In [ ]:
print(my_sixth_with_val_border_1.val)
print(my_sixth_with_val_border_3.val)

In the second val, there is not only 3 missing, but also 2. This is because the *val_border* property marks the *first* prime, which shall be ignored by the object, meaning it will also ignore all primes before (and including) the asked one. Setting the *val_border* of a pitch might be sometimes helpful, but if you only want to reduce all pitches to the same octave, it may be better to use the *normalize* - method, because setting the *val_border* will forbid any falling intervals, since - like the *normalize* - method - it will always reduce all pitches between 1/1 and *val_border*/1. You have to keep in mind that while you use the *normalize* - method once, the *val_border* of a pitch is permanent until you change it again.

### Using practically the idea of Monzo representations

Since we recognized, that we could represent pitches and intervals through vectors filled with positive and negative integers, we could also analyse pitches the same way we may analyse vectors. We could for example sum all integers (with their [absolute value](https://en.wikipedia.org/wiki/Absolute_value)) together and compare how many exponents an interval contains:

In [ ]:
ji.r(7, 6, val_border=2).summed()

Or we could shift the vector one number to the left or one number to the right; if we may have for example the interval 3/2 or in Monzo writing [-1, 1], we could shift this + 1 and we might get a Monzo with [0, -1, 1]. If we shift it -1 we get the Monzo [1] (numbers to the left are disappearing). If we set the *val_border* to 2, we could shift our perfect fifth to a major third:

In [ ]:
perfect_fifth = ji.r(3, 2, val_border=2)
major_third = perfect_fifth.shift(1)
pythagorean_scale_with_val_border2 = [pitch.set_val_border(2) for pitch in pythagorean_scale]
major_third_pythagorean_scale = [pitch.shift(1) for pitch in pythagorean_scale_with_val_border2]

*mu* offers different comparison methods between two pitches based on the Monzo model. They all start with *is_* and there are three different types of closeness: *is_related*, *is_congeneric* and *is_sibling*. While *is_related* only checks, whether the two asked pitches have at least on prime in common (no matter whether the exponent is positive or negative), *is_congeneric* only returns True if both pitches contain exactly the same primes (but how high or low the values of the exponents are doesn't matter again). The last method *is_related* does the same check like *is_congeneric* but to return True it also requires both pitches to contain the same *gender*.

In [ ]:
p0 = ji.JIPitch([0, 1, -1])
p1 = ji.JIPitch([2, 0, -3])
p2 = ji.JIPitch([0, -1, 1])
p3 = ji.JIPitch([0, 2, -2])
p0.is_related(p1) == True
p0.is_congeneric(p1) == False
p0.is_congeneric(p2) == True
p0.is_related(p2) == True
p0.is_sibling(p2) == False
p0.is_sibling(p3) == True

## Container for pitch objects 

Of course you could just save and collect your pitch - objects in Pythons [default data structures](https://docs.python.org/3/tutorial/datastructures.html) (like lists, tuples etc.), but *mu* also offers you some predefined container classes for pitch objects. While at first they may look like ordinary Python lists or sets, they offer additional methods which might help you to organise your pitches. The two most basic classes of these containers are the *JIMel* and the *JIHarmony* - class and you might already note their difference from their names: While *JIMel* helps you to organise *sequential* pitches, *JIHarmony* is a container for *simultan* pitches.

In [ ]:
my_mel = ji.JIMel([ji.r(1, 1), ji.r(3, 2), ji.r(5, 4)])
my_harmony = ji.JIHarmony([ji.r(1, 1), ji.r(3, 2), ji.r(5, 4)])

You can recognise their difference over many methods. For instance both of these objects have the property *intervals*, but...

In [ ]:
print(my_mel.intervals)  # return intervals between 1/1 and 3/2 and between 3/2 and 5/4
print(my_harmony.intervals)  # return intervals between 1/1 and 3/2 and between 3/2 and 5/4 and between 5/4 and 1/1

And while you could add to both container another *ji.r(3, 2)* *my_mel* will change, while *my_harmony* won't (because there can't be the same pitches twice in a harmony):

In [ ]:
my_mel.append(ji.r(3, 2))
my_harmony.add(ji.r(3, 2))  # note the difference: append (from list) vs. add (from set)
print(my_mel)
print(my_harmony)

Both are helpful to create more complicated *mu* objects. With *mu.sco.old* you could build Melodies (through *JIMel*) and Chords or Cadences (through *JIHarmony*). You could also use them to build scales. Let's try to make the second of [Johann Kirnbergers temperaments](https://en.wikipedia.org/wiki/Kirnberger_temperament)! I will follow these (German) [instructions](http://www.wolfgangkostujak.de/downloads/Stimmanweisungen.pdf) by [Wolfgang Kostujak](http://www.wolfgangkostujak.de/downloads/Stimmanweisungen.pdf). The basic idea is to start 3 times at your c4 and to add a couple of just intervals over this c. To archive this, I will make use of the *accumulate* - method of *ji.JIMel*. The idea of this method is to reinterpret pitches as intervals: so while the first element of your *JIMel* - object may be the beginning of your "Melody", the remaining elements are the intervals *between two pitches*, meaning that:

In [ ]:
my_melody = ji.JIMel([ji.r(1, 1), ji.r(3, 2), ji.r(5, 4)])
my_melody_intervals = my_melody.intervals
my_melody_intervals.insert(0, ji.r(1, 1))  # add the beginning of the melody
my_melody_intervals.accumulate() == my_melody

After understaning this method, we can start to create the actual scale:

In [ ]:
start = ji.r(1, 1)  # my c
kirnbergers2_pre = (
    ji.JIMel([start, ji.r(3, 2), ji.r(3, 2)]).accumulate(),  # two fifths from c
    ji.JIMel([start, ji.r(4, 3), ji.r(4, 3), ji.r(4, 3), ji.r(4, 3), ji.r(4, 3)]).accumulate(),  # and five fourth
    ji.JIMel([start, ji.r(5, 4), ji.r(3, 2), ji.r(3, 2)]).accumulate())  # one major third and two fifths
kirnbergers2 = ji.JIHarmony([])  # I will use JIHarmony to delete the double pitches (because of the many 'start's)
for sub in kirnbergers2_pre:  # adding all intervals to the kirnberger2 - object
    for pitch in sub:
        kirnbergers2.add(pitch)

Now there is only the a4 missing. While an actual Harpsichord or Piano tuner will tune the a4 by ear in respect to the already tuned e4 and d4 (he will try out to make a compromise between both), *mu* offers some help to find the best solution through math. For this we first have to figure out the perfect interval from d4 and the perfect interval from e4. Afterwards we calculate the distance between both intervals. Unfortunatly we can't just halve the resulting ratio because - like you may know - our ears don't hear pitch linear but logarithmically. Perhaps the best solution would be to use equal distances in terms of factors like it is common in equal temperaments. But then we lose access to many methods which the Just Intonation module of *mu* offers, due to which it might be valuable to find a compromise in Just Intonation. To find this compromise, we can make use of the *halve* - method of *mu*. This method tries to find a pair of two intervals which are as similar as possible and who multiplied result in the halved pitch object.

In [ ]:
d4 = ji.r(3, 2).scalar(2).normalize(2)
e4 = ji.r(5, 4)
perfect_a_from_d = d4 + ji.r(3, 2)
perfect_a_from_e = e4 + ji.r(4, 3)
if perfect_a_from_d.float > perfect_a_from_e.float:
    difference = perfect_a_from_d - perfect_a_from_e
else:
    difference = perfect_a_from_e - perfect_a_from_d
halved = difference.halve(allowed_primes=[2, 3, 5], max_depth=8)
if perfect_a_from_d.float > perfect_a_from_e.float:
    compromise_a = halved[0] + perfect_a_from_e
else:
    compromise_a = halved[0] + perfect_a_from_d
kirnberger2.add(compromise_a)

After we created our Kirnberger II temperament now, we could save it in a proper *Scale* - object. To create a *JIScale* two arguments are necessary: one for the actual pitches and the second one for the *frame size*. The *frame size* is comparable to the *val_border* or the *normalize* - method we already saw. It just marks when the scale should repeat itself again.

In [ ]:
kirnberger2_scale = ji.JIScale(kirnberger2, ji.r(2, 1))

Now we could try to calculate the harmonicity of the whole scale! I think it should be as harmonic as the intervals between the different pitches of the scale are. For this we write a small function, which return the median of all harmonicity values in the scale. For being able to try out different harmonicity functions, we add the optional *harmonicity_function* - argument.

In [ ]:
def calculate_harmonicity_of_scale(scale, harmonicity_function=lambda p: p.harmonicity_euler):
    intervals = scale.intervals
    harmonicities = sorted(tuple(harmonicity_function(interval) for interval in intervals))
    median_harmonicity = harmonicities[len(harmonicities) // 2]
    return median_harmonicity

kirnberger2_harmonicity = calculate_harmonicity_of_scale(kirnberger2_scale)

After rebuilding this historic tuning, we could go further to more recent developments: let's check out La Monte Youngs scale for his [Well Tuned Piano](https://en.wikipedia.org/wiki/The_Well-Tuned_Piano):

In [ ]:
young = ji.JIScale([
    ji.r(1, 1),
    ji.r(567, 512),
    ji.r(9, 8),
    ji.r(147, 128),
    ji.r(21, 16),
    ji.r(1323, 1024),
    ji.r(189, 128),
    ji.r(3, 2),
    ji.r(49, 32),
    ji.r(7, 4),
    ji.r(441, 256),
    ji.r(63, 32)
], ji.r(2, 1))

If we analyse the used primes, we can see there are only 2, 3 and 7. For some reasons Mr. Young seems to avoid the 5 (major third / minor sixth), but compared to historic tunings he added (like Harry Partch) the 7:

In [ ]:
print(young.primes)

If we set the *val_border* to 2, we could also find out the most often used prime (apart from 2):

In [ ]:
young.val_border = 2
print(young.dominant_prime)

## Exporting and importing pitch objects

Sometimes you might want to save your pitches to your computer to recall them after you restarted your Python interpreter. Currently you can save your pitch objects with *mu* to the [json-format](https://www.json.org/index.html). In the future *mu* may also support the famous [scl-format](http://www.huygens-fokker.org/scala/scl_format.html) of [Scala](http://www.huygens-fokker.org/scala).
To save a pitch or JIContainer to a *.json* - file, you can easily use the *export2json* - method:

In [ ]:
my_unforgetable_mel_object = ji.JIMel([ji.r(7, 4), ji.r(8, 7), ji.r(1, 1)])
my_unforgetable_mel_object.export2json("beautiful_scale132.json")

To import an object you may use the *load_json* method:

In [ ]:
my_forgetable_mel_object = ji.JIMel.load_json("beautiful_scale132.json")
my_forgetable_mel_object == my_unforgetable_mel_object

## Exploring *mu.mel.ji*

There are many more methods and functions to explore in the *ji* - module. If you use [IPython](https://ipython.org/) you could scroll through the different methods and try them out or read their documentation through Pythons *help* function (for instance: help(ji.JIPitch.shift)).